In [1]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from mcp_persor import BVHparser


In [2]:
# 窓ごとの特徴量を抽出
## データの読み込み
bvhp = BVHparser('./logs/refrigerator.bvh')
df = bvhp.get_motion_df()

# 窓ごとの特徴量を抽出する関数
def calculate_features(window):
    features = []
    
    # 中央値の計算
    median_time = window['time'].mean()
    
    # 平均、分散、相関、ゼロ交差数、エネルギー、エントロピーの計算
    for joint in columns:
        X_rotate = window[joint + '_Xrotation'].values
        Y_rotate = window[joint + '_Yrotation'].values
        Z_rotate = window[joint + '_Zrotation'].values
        
        # 平均
        mean_value = np.mean([X_rotate, Y_rotate, Z_rotate])
        # 分散
        variance_value = np.var([X_rotate, Y_rotate, Z_rotate])
        # 相関
        correlation_value = np.corrcoef([X_rotate, Y_rotate, Z_rotate])[0, 1]
        # ゼロ交差数
        zero_crossings = np.sum(np.diff(np.signbit([X_rotate, Y_rotate, Z_rotate])) != 0)
        # エネルギー
        energy_value = np.sum(np.square([X_rotate, Y_rotate, Z_rotate]))
        # エントロピー
        # entropy_value = entropy([X_rotate, Y_rotate, Z_rotate])
        
        # features.extend([mean_value, variance_value, correlation_value, zero_crossings, energy_value, entropy_value])
        features.extend([mean_value, variance_value, correlation_value, zero_crossings, energy_value])
    
    return [median_time] + features

# 窓ごとの特徴量を計算し、結果をデータフレームに格納
window_size = 3
window_stride = 1

columns = [j for j in bvhp.get_joints() if not j.startswith('_')]
windows = [df.iloc[i:i + window_size] for i in range(0, len(df), window_stride) if i + window_size <= len(df)]
# result_columns = ['time'] + [f'{joint}_{metric}' for joint in columns for metric in ['mean', 'var', 'corr', 'zero_cross', 'energy', 'entropy']]
result_columns = ['time'] + [f'{joint}_{metric}' for joint in columns for metric in ['mean', 'var', 'corr', 'zero_cross', 'energy']]
result_data = [calculate_features(window) for window in windows]
result_df = pd.DataFrame(result_data, columns=result_columns)

# ラベル情報をもとにstateカラムを追加
labels = [
    {'title': '冷蔵庫を開ける', 'num': 1, 'times': [[10, 12], [33, 35], [39, 41], [55, 57], [77.5, 79.5], [104, 106], [133, 135]]},
    {'title': '冷蔵庫を閉める', 'num': 2, 'times': [[14, 16], [36, 38], [42, 45], [58, 60], [82, 84], [106.5, 108.5], [135, 137]]},
]

result_df['state'] = 0  # 初期値として全ての行に0を設定

for label in labels:
    for time_range in label['times']:
        start_time, end_time = time_range
        result_df.loc[(result_df['time'] >= start_time) & (result_df['time'] <= end_time), 'state'] = label['num']

# 結果をCSVファイルに書き出し
result_df.to_csv('./logs/refrigerator_with_state.csv', index=False)


/Users/satooru/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/satooru/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
